# House Prices - Advanced Regression Techniques (Kaggle)

Run after following notebooks are run:
1. 01 Data Cleaning
2. 02 Exploratory Data Analysis

#### Development Ideas
* 

In [12]:
### set up libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import KFold, cross_val_score
import seaborn as sns

pd.options.display.max_rows = 100

## Load Data

In [7]:
### load data (cleaned)
train = pd.read_csv('train_clean_truncated.csv')

### convert all object datatypes to category
for col in train.select_dtypes(include='object'):
    train[col] = train[col].astype('category')

### convert additional columns to category datatype
train['MSSubClass'] = train['MSSubClass'].astype('category')

train.shape

(1460, 67)

In [9]:
## separate in to predictor and response tables (x and y)
x_train = train.drop('SalePrice', axis=1)
y_train = train.SalePrice

## One Hot Encoding

In [21]:
x_train_onehot = pd.get_dummies(x_train, drop_first=True)
print(x_train_onehot.shape)
x_train_onehot.head()

(1460, 201)


,LotFrontage,LotArea,OverallQual,OverallCond,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,...,PavedDrive_P,PavedDrive_Y,Fence_GdWo,Fence_MnPrv,Fence_MnWw,SaleType_Other,SaleType_WD,SaleCondition_Other,SaleCondition_Partial,remodel_flag_Y
0,0.272948,0.348136,7.0,5.0,0.868041,0.560897,0.0,0.255764,0.232680,0.380229,...,False,True,False,True,False,False,True,False,False,False
1,0.341128,0.372612,6.0,8.0,0.000000,0.616690,0.0,0.354284,0.310501,0.529079,...,False,True,False,True,False,False,True,False,False,False
2,0.287200,0.403172,7.0,5.0,0.852437,0.501901,0.0,0.438264,0.245498,0.408222,...,False,True,False,True,False,False,True,False,False,True
3,0.248403,0.371609,7.0,5.0,0.000000,0.390005,0.0,0.488573,0.212140,0.425073,...,False,True,False,True,False,False,True,True,False,True
4,0.358142,0.449134,8.0,5.0,0.911389,0.548640,0.0,0.465562,0.288850,0.492198,...,False,True,False,True,False,False,True,False,False,False


## Modelling - Cross-Validation and Optimisation

### KNN

In [22]:
## instantiate knn algorithm
knn = KNeighborsRegressor(n_neighbors=10)

## cross validation
kf = KFold(n_splits=5, shuffle=True, random_state=8)
cv_scores = cross_val_score(knn, x_train_onehot.values, y_train.values, cv=kf)
cv_scores

array([0.63370964, 0.6258547 , 0.5807764 , 0.67503709, 0.6325129 ])